# <center>Первая модель по решению задачи классификации</center>


Базанов Дмитрий 
Б03-903

 ## 1. Импортируем все нужное

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
import warnings
warnings.filterwarnings('ignore') 
#Убирает все варнинги, возникающие при использовании логистиечской регресии)

## 2. Загрузим файл

In [7]:
data1 = pd.read_csv('part1_churn.csv')
data2 = pd.read_csv('part2_churn.csv')

In [8]:
data1.shape

(7073, 20)

In [9]:
data2.shape

(7087, 2)

### 2.1 Удалим  из второго фрейма последние 14 ячеек. Это единственный выход, так как иначе будут ошибки при обучении модели.

In [10]:
data2.drop(range(7073, 7087, 1), inplace=True)

In [11]:
print(data1.shape[0])
print(data2.shape[0])

7073
7073


## 3. Сделаем подготовку данных, аналогичную первому заданию

In [12]:
data_new=data1.merge(data2, on = 'customerID', how = 'right')
data_new = data_new.drop_duplicates().reset_index(drop=True)

In [13]:
data_new.tail(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


## 4. Заполним пробелы в данных,  проведём label-encoding

### 4.1 Применим импутацию всех null значений их средними(по всем данным, кроме типа float)

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
my_imputer = SimpleImputer()
data_obj = data_new.select_dtypes(include=['object'])
data_flt = data_new.select_dtypes(exclude=['object'])
data_inputed = pd.DataFrame(my_imputer.fit_transform(data_flt))
data_inputed.columns = data_flt.columns
data_new1=data1.merge(data2, on = 'customerID', how = 'right')
data_new1.head(3)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


 ###  4.2 Теперь Label-encoding, так как  у нас много категориальных признаков

In [16]:
# Создадим список категориальных и некатегориальных фич.
data_cat = ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling' , 'Churn',
       'PaymentMethod'] 
data_var=['customerID', 'SeniorCitizen', 'tenure','MonthlyCharges', 'TotalCharges'] 
#Здесь в отлииче от первого задания сразу имеем возможность изменить churn с категориального признака на 
# численный без лямбла функции


In [17]:
data_enc_obj = data_new1[data_cat]
data_enc_flt = data_new1[data_var]
data_cat.remove('customerID')

In [18]:
data_enc_obj.head(3)

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,Churn,PaymentMethod
0,7590-VHVEG,Female,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,No,Electronic check
1,5575-GNVDE,Male,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,No,Mailed check
2,3668-QPYBK,Male,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Yes,Mailed check


In [19]:
data_enc_flt.head(3)

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,7590-VHVEG,0,1,29.85,29.85
1,5575-GNVDE,0,34,56.95,1889.5
2,3668-QPYBK,0,2,53.85,108.15


In [20]:
from sklearn.preprocessing import LabelEncoder

data_enc_obj_labled = data_enc_obj.copy()
label_encoder = LabelEncoder()
for col in data_cat:
    data_enc_obj_labled[col] = label_encoder.fit_transform(data_enc_obj[col])


In [21]:
 data_enc_obj_labled.head(3)

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,Churn,PaymentMethod
0,7590-VHVEG,0,1,0,0,1,0,0,2,0,0,0,0,0,1,0,2
1,5575-GNVDE,1,0,0,1,0,0,2,0,2,0,0,0,1,0,0,3
2,3668-QPYBK,1,0,0,1,0,0,2,2,0,0,0,0,0,1,1,3


### 4.3 Мёрджим с некатегориальными признаками


In [22]:
data_encoded=data_enc_obj_labled.merge(data_enc_flt, on = 'customerID', how = 'right')

In [23]:
data_encoded.head(3)

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,Churn,PaymentMethod,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,7590-VHVEG,0,1,0,0,1,0,0,2,0,...,0,0,0,1,0,2,0,1,29.85,29.85
1,5575-GNVDE,1,0,0,1,0,0,2,0,2,...,0,0,1,0,0,3,0,34,56.95,1889.5
2,3668-QPYBK,1,0,0,1,0,0,2,2,0,...,0,0,0,1,1,3,0,2,53.85,108.15


### 4.4 В ходе уже обучения модели возникли непредвиденные проблемы с данными, они исправляются в этом блоке

In [24]:
data_encoded["TotalCharges"] = pd.to_numeric(data_encoded["TotalCharges"], errors='coerce');
data_encoded.drop([ 'customerID'], axis=1, inplace=True) 

In [25]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [26]:
clean_dataset(data_encoded);

In [27]:
y =  data_encoded.Churn
data_encoded.drop(['Churn'], axis=1, inplace=True)

## 5. После всей подготовки разделим данные на тренировочную и тестовую выборки

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(data_encoded, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

## 6. Применим первую модель(рассмотрим 2 модели для задач классификации)

In [29]:
from sklearn import model_selection, linear_model, metrics

### 6.1 RidgeClassifier

In [30]:
ridge_classifier = linear_model.RidgeClassifier(random_state=1) # Не будем настраивать параметры модели,
#обучим с параметрами по умолчанию

In [31]:
ridge_classifier.fit(X_train, y_train)

RidgeClassifier(random_state=1)

In [32]:
ridge_predictions = ridge_classifier.predict(X_valid)

In [37]:
def metrics_all(y, predictions):
    print('Accuracy: {:.3f}'.format( metrics.accuracy_score(y, predictions)))
    print('Recall: {:.3f}'.format(metrics.recall_score(y, predictions)))
    print('F1: {:.3f}'.format( metrics.f1_score(y, predictions)))
    print('Logloss: {:.3f}'.format( metrics.log_loss(y, predictions)))


In [38]:
metrics_all(y_valid, ridge_predictions)

Accuracy: 0.806
Recall: 0.496
F1: 0.571
Logloss: 6.707


In [39]:
ridge_classifier.coef_ 

array([[-9.03286324e-03, -1.39599772e-02, -3.76840540e-02,
        -2.90443686e-01,  2.72670601e-02,  8.84144649e-02,
        -9.13564029e-02, -5.08309438e-02, -3.23846151e-02,
        -8.76531922e-02,  3.38476980e-03,  3.17503496e-03,
        -9.30313845e-02,  1.11790620e-01,  1.70588155e-02,
         6.62625772e-02, -3.09609714e-03,  1.13492856e-02,
        -1.00475789e-04]])

In [40]:
ridge_classifier.intercept_ #коэффициент регуляризации

array([-0.51958561])

### 6.2 LogisticRegression

In [41]:
log_regressor = linear_model.LogisticRegression(random_state=1)

In [42]:
log_regressor.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [43]:
lr_predictions = log_regressor.predict(X_valid)

In [44]:
lr_proba_predictions = log_regressor.predict_proba(X_valid)

In [45]:
metrics_all(y_valid, lr_predictions)

Accuracy: 0.807
Recall: 0.560
F1: 0.601
Logloss: 6.683


In [46]:
lr_proba_predictions

array([[0.82800241, 0.17199759],
       [0.62088972, 0.37911028],
       [0.75487261, 0.24512739],
       ...,
       [0.95042382, 0.04957618],
       [0.99415458, 0.00584542],
       [0.97438237, 0.02561763]])

## 7. Теперь попробуем поработать с кросс-валидацией и улучшить показания метрик

### 7.1 Проводим кросс-валидацию сразу для 2-х классификаторов на разных метриках

In [47]:
metrics_list=['accuracy', 'recall', 'f1', 'roc_auc', 'precision']
#Список метрик, которые передадим в функцию

In [73]:
def cv_metrics(metrics_list, model, data_encoded, y, n): #Функция вычисления разных параметров метрик на выборке
    for i in metrics_list:
        scoring =  model_selection.cross_val_score(model, data_encoded, y,  scoring=i, cv=n)
        print('{} mean:{:.3f}, max:{:.3f}, min:{:.3f}, std:{:.3f}'.format(i, scoring.mean(), scoring.max(), scoring.min(), scoring.std()))

### 7.2 Пример использования без функций

In [68]:
ridge_scoring = model_selection.cross_val_score(ridge_classifier, data_encoded, y,  scoring='recall', cv=10) 

In [69]:
lr_scoring = model_selection.cross_val_score(log_regressor, data_encoded, y,  scoring='recall', cv=10);

In [72]:
print('Ridge mean:{:.3f}, max:{:.3f}, min:{:.3f}, std:{:.3f}'.format(ridge_scoring.mean(), ridge_scoring.max(), 
                                              ridge_scoring.min(), ridge_scoring.std()))

Ridge mean:0.506, max:0.594, min:0.448, std:0.040


In [71]:
print('LR mean:{:.3f}, max:{:.3f}, min:{:.3f}, std:{:.3f}'.format(lr_scoring.mean(), lr_scoring.max(), 
                                              lr_scoring.min(), lr_scoring.std()))

LR mean:0.557, max:0.630, min:0.516, std:0.031


### 7.3 Запустимся с функцией

In [74]:
cv_metrics(metrics_list, ridge_classifier, data_encoded, y, 20)

accuracy mean:0.800, max:0.859, min:0.767, std:0.021
recall mean:0.505, max:0.639, min:0.423, std:0.058
f1 mean:0.573, max:0.709, min:0.494, std:0.051
roc_auc mean:0.835, max:0.881, min:0.787, std:0.023
precision mean:0.664, max:0.795, min:0.586, std:0.048


In [75]:
cv_metrics(metrics_list, log_regressor, data_encoded, y, 20)

accuracy mean:0.802, max:0.853, min:0.769, std:0.022
recall mean:0.563, max:0.656, min:0.448, std:0.057
f1 mean:0.602, max:0.701, min:0.509, std:0.047
roc_auc mean:0.841, max:0.885, min:0.780, std:0.026
precision mean:0.650, max:0.775, min:0.573, std:0.050


## 8. Наконец воспользуемся сеткой, чтоб подобрать лучшие параметры, например, accuracy. 

In [76]:
classifier = linear_model.SGDClassifier(random_state=0)

In [59]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [77]:
parametrs_grid = {
    'loss': ['hinge', 'squared_loss'], #Функции потерь
    'penalty': ['l1', 'l2'], #тип регулязации 
    'alpha' : np.linspace(0.0001, 0.001, num=5) #значение константы релуяризатора
    
}

In [78]:
cv = model_selection.StratifiedShuffleSplit(test_size=0.2,  n_splits= 10, random_state=0)

In [79]:
grid_cv = model_selection.GridSearchCV(estimator=classifier, param_grid=parametrs_grid, scoring = 'accuracy', cv=cv)

In [80]:
%%time
grid_cv.fit(data_encoded, y)

Wall time: 32.7 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'squared_loss'],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [81]:
grid_cv.best_estimator_ #Лучшая настройка классицикатора

SGDClassifier(alpha=0.000325, penalty='l1', random_state=0)

In [83]:
grid_cv.best_score_ 

0.7642163661581136

In [82]:
grid_cv.best_params_

{'alpha': 0.000325, 'loss': 'hinge', 'penalty': 'l1'}